In [43]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.statespace.sarimax import SARIMAX
import re

# Load datasets with specified encoding

merged_df = pd.read_csv('merged_dataset.csv')


# Recreate the Date column in the water quality dataset
merged_df['Date'] = pd.to_datetime(merged_df['Year'].astype(str) + '-' + merged_df['Month'].astype(str), errors='coerce')


# Create lag features
lags = [1, 2, 3]
parameters = ['pH (units)', 'Ammonia (mg/L)', 'Nitrate (mg/L)', 'Inorganic Phosphate (mg/L)', 'BOD (mg/l)', 'Dissolved Oxygen (mg/l)', 'Total coliforms (MPN/100ml)']

for parameter in parameters:
    for lag in lags:
        merged_df[f'{parameter}_lag_{lag}'] = merged_df[parameter].shift(lag)

# Drop rows with NaN values resulting from lag feature creation
merged_df.dropna(inplace=True)




# Select relevant columns for normalization
cols_to_normalize = parameters + ['Temperature', 'Dew Point', 'Humidity', 'Wind Speed', 'Pressure', 'Precip.']

# Normalize the selected columns
scaler = StandardScaler()
merged_df[cols_to_normalize] = scaler.fit_transform(merged_df[cols_to_normalize])

# Set Date as index for merged_df
merged_df.set_index('Date', inplace=True)

# Define and fit the SARIMA model for each parameter
sarima_order = (1, 1, 1)
seasonal_order = (1, 1, 1, 12)

models = {}
forecasts = {}

for parameter in parameters:
    model = SARIMAX(merged_df[parameter], order=sarima_order, seasonal_order=seasonal_order)
    model_fit = model.fit(disp=False)
    forecast = model_fit.forecast(steps=1)
    models[parameter] = model_fit
    forecasts[parameter] = forecast

# Print the forecasts
for parameter, forecast in forecasts.items():
    print(f"Forecast for {parameter}:")
    print(forecast)

merged_df.columns


C:\Users\John Wilson\AppData\Local\Temp\ipykernel_21976\3959118802.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_df['Date'] = pd.to_datetime(merged_df['Year'].astype(str) + '-' + merged_df['Month'].astype(str), errors='coerce')
E:\Anaconda\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
E:\Anaconda\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
E:\Anaconda\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results wil

Forecast for pH (units):
489   -1.473098
dtype: float64
Forecast for Ammonia (mg/L):
489    0.185631
dtype: float64
Forecast for Nitrate (mg/L):
489    1.01014
dtype: float64
Forecast for Inorganic Phosphate (mg/L):
489   -0.135562
dtype: float64
Forecast for BOD (mg/l):
489   -0.656565
dtype: float64
Forecast for Dissolved Oxygen (mg/l):
489   -0.318767
dtype: float64
Forecast for Total coliforms (MPN/100ml):
489   -0.189839
dtype: float64


E:\Anaconda\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
E:\Anaconda\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Index(['Monitoring Stations', 'Month', 'Year', 'pH (units)', 'Ammonia (mg/L)',
       'Nitrate (mg/L)', 'Inorganic Phosphate (mg/L)', 'BOD (mg/l)',
       'Dissolved Oxygen (mg/l)', 'Total coliforms (MPN/100ml)',
       'Phytoplankton (cells/ml)', 'Wind', 'Condition', 'Time', 'Temperature',
       'Dew Point', 'Humidity', 'Wind Speed', 'Wind Gust', 'Pressure',
       'Precip.', 'pH (units)_lag_1', 'pH (units)_lag_2', 'pH (units)_lag_3',
       'Ammonia (mg/L)_lag_1', 'Ammonia (mg/L)_lag_2', 'Ammonia (mg/L)_lag_3',
       'Nitrate (mg/L)_lag_1', 'Nitrate (mg/L)_lag_2', 'Nitrate (mg/L)_lag_3',
       'Inorganic Phosphate (mg/L)_lag_1', 'Inorganic Phosphate (mg/L)_lag_2',
       'Inorganic Phosphate (mg/L)_lag_3', 'BOD (mg/l)_lag_1',
       'BOD (mg/l)_lag_2', 'BOD (mg/l)_lag_3', 'Dissolved Oxygen (mg/l)_lag_1',
       'Dissolved Oxygen (mg/l)_lag_2', 'Dissolved Oxygen (mg/l)_lag_3',
       'Total coliforms (MPN/100ml)_lag_1',
       'Total coliforms (MPN/100ml)_lag_2',
       'Total coli